In [1]:
import pandas as pd
from pvtools import Sequence
import json
import numpy as np

In [2]:
tg1 = Sequence(fasta_file='TestGene1.fasta', json_file='TestGene1.json')

In [3]:
tg1.to_tsv('TestGene1.tsv')

In [4]:
tg1.get_cds_dataframe()

,Name,Start,End
0,Upstream,1,100
1,5' UTR Exon 1,101,200
2,5' UTR Intron 1,201,300
3,5' UTR Exon 2,301,350
4,CDS 1,351,400
5,Intron 1,401,500
6,CDS 2,501,600
7,Intron 2,601,700
8,CDS 3,701,800
9,Intron 3,801,900


In [ ]:
ng = Sequence(fasta_file='NG_011745.1.fasta', json_file='NG_011745.1.json')
nm = Sequence(fasta_file='NM_006446.5.fasta', json_file='NM_006446.5.json')

In [ ]:
ng.to_tsv('tabel.tsv')

In [ ]:
df = ng.get_cds_dataframe()
df

In [ ]:
113603 - 112997

In [ ]:
# for pos, ref in ng_dict.items():
#     print(ng.data[pos-1] == ref)

In [ ]:
# with open('NM_006446.5.json', 'w') as f:
#     json.dump(data, f, indent=4)

In [ ]:
a = [1,2,3]
a + 1

# Test 1.

In [ ]:
df = pd.read_table('manual.tsv')
df = df[df['NG_011745.1 (start=1)'] != '.']
df['Ref'] = df['SNP'].str.split('>').str[0]
df['NG_011745.1 (start=1)'] = df['NG_011745.1 (start=1)'].astype(int)
ng_dict = dict(zip(df['NG_011745.1 (start=1)'], df['Ref']))
results = []
for pos, allele in ng_dict.items():
    results.append(ng.seq[pos-1] == allele)
if all(results):
    print('All alleles passed')

# Test 2.

In [ ]:
ng.transcribe() == nm.seq

# Test 3.

In [ ]:
df = pd.read_table('manual.tsv')

b = []

for item in df['Impact'].to_list():
    if any(x.isdigit() for x in item):
        b.append(item)
        
pos_list = []

for item in b:
    pos_list.append(int(''.join([x for x in item if x.isdigit()])))

ref_aa = []

for pos in pos_list:
    ref_aa.append(ng.data['Protein'][pos-1])

for i in range(len(b)):
    item = b[i]
    pos = pos_list[i]
    aa = item.split(str(pos))[0]
    print(aa == ref_aa[i], aa, ref_aa[i], pos)

In [ ]:
4195 - 5001 - 105 + 1

In [ ]:
5001 - 833 + 105 -1

In [ ]:
5001 - 910 + 105 -1

In [ ]:
105, 2180

In [ ]:
ng.liftover(4883)


In [ ]:
ng.data['CDSEnds'][0] - ng.data['CDSStarts'][0]

In [ ]:
ng._get_start_to_atg()

In [ ]:
bp_sum

In [ ]:
bp_sum = 0

for i in range(b-1):
    start = data['FASTA']['NG_011745.1']['mRNAStarts'][i]
    end = data['FASTA']['NG_011745.1']['mRNAEnds'][i]
    bp_sum = end - start + 1